In [1]:
import pyautogui
import os, sys
import cv2
import numpy as np
import math
import pandas as pd
import warnings
from progressbar import ProgressBar
from pathlib import Path
import joblib
from sklearn.decomposition import PCA
from sklearn import svm, datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier  
from sklearn.ensemble import VotingClassifier
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from datetime import datetime
from flask import Flask, flash, request, redirect, url_for, render_template, Response
from werkzeug.utils import secure_filename
from sklearn.preprocessing import StandardScaler as sc

In [2]:
pbar = ProgressBar()

x_train = []
y_train = []
X = []
Y = []
name_grp = []
data_fr = []
G = []
pca_mean = 0

In [12]:
class FaceRecognition:
    person_no = 1  
    def __init__(self):
        self.encode_faces()
        self.train()
        # self.test("Vasu beladiya_2.jpg")
    
    def encode_faces(self):
        global name_grp,X,Y,data_fr,pca_mean          
        faces = {}
        name_grp = set()
        for image in pbar(os.listdir('Faces')):
            person_name = image.split("_")[0]            
            self.person_no+=1;
            faces[image] = cv2.imdecode(np.fromfile(f"Faces\{image}", dtype=np.uint8), cv2.IMREAD_COLOR)
            X.append(faces[image].flatten())
            Y.append(person_name)
            name_grp.add(person_name)
        # print(len(X),len(X[0]))
        # X = np.array(X)
        pca = PCA(n_components=40).fit(X)
        X = pca.transform(X)
        joblib.dump(pca,'models\\PCA.pkl')
    #     n_components = 128
    #     eigenfaces = pca.components_[:n_components]
    #     X = eigenfaces@(X - pca.mean_).T
    #     X = X.T
        name_grp = list(name_grp) 
        joblib.dump(name_grp,'models\\names.pkl')
        print("Data fetched Successfully..")  
        data_fr = pd.DataFrame(X)
        temp = []
        for k in Y:
            temp.append(name_grp.index(k)+1)
        Y = temp
        data_fr['y'] = Y
        new_data = data_fr.copy()
        new_data = new_data.sample(frac = 1)
        filepath = Path('data.csv')  
        # filepath.parent.mkdir(parents=True, exist_ok=True)  
        new_data.to_csv(filepath,index=False,header=False) 
        print("Data Saved..")
        # print(data_fr)
        # print(y_train)

    def train(self):
        global data_fr,X,Y
        data_fr = pd.read_csv("data.csv")
        Y = data_fr.iloc[:,-1]
        X = data_fr.iloc[:,:-1]
        print(X.shape)
        print(Y.shape)

        
        knn = KNeighborsClassifier()
        knn_params = dict(n_neighbors=list(range(10,30)))
        grid_knn = GridSearchCV(knn, knn_params, cv=5, scoring='accuracy', return_train_score=False)
        grid_knn.fit(X, Y)
        joblib.dump(grid_knn,'models\\KNN.pkl')
        print("KNN trained")        

        clf = svm.SVC(probability=True)

        clf_params={
            "C":[0.001,0.01,0.0001],
            "gamma":[0.001,0.01,0.0001],
            "kernel":["rbf"]
        }
        grid_svc = GridSearchCV(clf,clf_params,refit=True,verbose=3)
        grid_svc.fit(X,Y)
    #         SVC_params = grid.best_params_()
        joblib.dump(grid_svc,'models\\SVC.pkl')
        print("Support Vector Classifier Trained")


        
        RF_classifier= RandomForestClassifier()  
        RF_classifier.fit(X,Y)  
        RF_params = { 
            'n_estimators': [5, 20],
            'max_features': ['auto', 'sqrt', 'log2'],
            'max_depth' : [4,5,6,7,8],
            'criterion' :['gini', 'entropy']
        }
        grid_RF = GridSearchCV(RF_classifier, RF_params, cv= 5)
        joblib.dump(grid_RF,'models\\RF.pkl')
        print("Random Forest Classifier Trained")

    #         classifier.

        grid={'hidden_layer_sizes':[70,50,40], 'activation':['relu','tanh'], 'solver':['adam'], 'learning_rate' : ['constant'],'learning_rate_init':[0.01,0.001],'max_iter':[6000,2000,4000]}
        
        # nn = MLPClassifier(hidden_layer_sizes=(10),activation="relu",solver="adam",learning_rate="constant",learning_rate_init=0.001,max_iter=1000)
        nn=GridSearchCV(MLPClassifier(), grid, n_jobs=-1)
        nn.fit(X,Y)

    #         MLP_params = nn.get_params()      
        joblib.dump(nn,'models\\MPL.pkl')

        print("MultiLayer Protocol Trained")

        

        ensbl = VotingClassifier(estimators = [('knn', knn), ('svc', grid_svc),('rf',RF_classifier),('ANN',nn)],voting='soft')
        ensbl.fit(X,Y)        
        joblib.dump(ensbl,'models\\ENSBL.pkl')
        print("Ensemble Trained")
        
        epoch = 10
        n_dimensions = data_fr.shape[1]-1
        Acc = []
        # n_datapoints = result.shape[0]

        for e in range(epoch):

            train_data = data_fr.sample(frac=0.80)

            train_data = train_data.values
            train_x = train_data[:,:-1]
            train_y = train_data[:,-1].ravel()

            test_data = data_fr.sample(frac=0.20)
            test_data = test_data.values
            test_x = test_data[:,:-1]
            test_y = test_data[:,-1].ravel()

            ensbl.fit(train_x,train_y)
            y_pred = ensbl.predict(test_x)
            Acc.append(accuracy_score(test_y,y_pred))
        for ac in Acc:
            print(ac)
        print("-"*20)
        print("accuracy_score : {:.5f}".format(np.sum(Acc)/epoch))

fr = FaceRecognition()

100% |########################################################################|


Data fetched Successfully..
Data Saved..
(2561, 40)
(2561,)
KNN trained
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ..C=0.001, gamma=0.001, kernel=rbf;, score=0.047 total time=   3.6s
[CV 2/5] END ..C=0.001, gamma=0.001, kernel=rbf;, score=0.047 total time=   3.7s
[CV 3/5] END ..C=0.001, gamma=0.001, kernel=rbf;, score=0.047 total time=   3.7s
[CV 4/5] END ..C=0.001, gamma=0.001, kernel=rbf;, score=0.047 total time=   3.7s
[CV 5/5] END ..C=0.001, gamma=0.001, kernel=rbf;, score=0.047 total time=   3.7s
[CV 1/5] END ...C=0.001, gamma=0.01, kernel=rbf;, score=0.047 total time=   3.7s
[CV 2/5] END ...C=0.001, gamma=0.01, kernel=rbf;, score=0.047 total time=   3.7s
[CV 3/5] END ...C=0.001, gamma=0.01, kernel=rbf;, score=0.047 total time=   3.7s
[CV 4/5] END ...C=0.001, gamma=0.01, kernel=rbf;, score=0.047 total time=   3.7s
[CV 5/5] END ...C=0.001, gamma=0.01, kernel=rbf;, score=0.047 total time=   3.9s
[CV 1/5] END .C=0.001, gamma=0.0001, kernel=rbf;, score=0.